## Download data

In [1]:
!wget -O ./0_raw_data/CNV.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FGistic2_CopyNumber_Gistic2_all_data_by_genes.gz
!wget -O ./0_raw_data/Methyl.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FHumanMethylation27.gz
!wget -O ./0_raw_data/mRNA.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FHT_HG-U133A.gz
!wget -O ./0_raw_data/RNAseq.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FHiSeqV2_PANCAN.gz

--2022-03-06 16:30:38--  https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FGistic2_CopyNumber_Gistic2_all_data_by_genes.gz
Resolving tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)... 52.217.161.10
Connecting to tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)|52.217.161.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633969 (1.6M) [binary/octet-stream]
Saving to: './0_raw_data/CNV.gz'

100%[======================================>] 1,633,969   --.-K/s   in 0.09s   

2022-03-06 16:30:38 (17.3 MB/s) - './0_raw_data/CNV.gz' saved [1633969/1633969]

--2022-03-06 16:30:39--  https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FHumanMethylation27.gz
Resolving tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)... 52.217.161.10
Connecting to tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.

## Read data into DataFrame

In [2]:
!gunzip ./0_raw_data/*.gz

In [3]:
import pandas as pd

df_list = {}
data_cnv_file = "./0_raw_data/CNV"
data_mRNA_file = "./0_raw_data/mRNA"
data_RNAseq_file = "./0_raw_data/RNAseq"
data_methyl_file = "./0_raw_data/Methyl"

df_list["cnv"] = pd.read_csv(data_cnv_file, sep="\t", index_col=0)
df_list["mrna"] = pd.read_csv(data_mRNA_file, sep="\t", index_col=0)
df_list["rnaseq"] = pd.read_csv(data_RNAseq_file, sep="\t", index_col=0)
df_list["methyl"] = pd.read_csv(data_methyl_file, sep="\t", index_col=0)

for key in df_list.keys():
    print(f"{key} \t {df_list[key].shape}")

cnv 	 (24776, 579)
mrna 	 (12042, 593)
rnaseq 	 (20530, 308)
methyl 	 (27578, 616)


## Intersections

### Intersection \#1: CNV + mRNA + DNA methylation

In [4]:
col_idx = pd.concat([df_list["cnv"], df_list["mrna"], df_list["methyl"]], axis=0, join="inner").columns
print(len(col_idx))

555


In [5]:
df_list["cnv_1"] = df_list["cnv"][col_idx]
df_list["mrna_1"] = df_list["mrna"][col_idx]
df_list["methyl_1"] = df_list["methyl"][col_idx]

### Intersection \#2: CNV + RNAseq + DNA methylation

In [6]:
col_idx = pd.concat([df_list["cnv"], df_list["rnaseq"], df_list["methyl"]], axis=0, join="inner").columns
print(len(col_idx))

292


In [7]:
df_list["cnv_2"] = df_list["cnv"][col_idx]
df_list["rnaseq_2"] = df_list["rnaseq"][col_idx]
df_list["methyl_2"] = df_list["methyl"][col_idx]

## Zero/Missing/NA values removal

In [8]:
def remove_missing_zero_data(df):
    df = df.dropna(axis=0, how='any')
    df = df[df.where(df != 0).any(axis=1)]
    return df

In [9]:
import numpy as np

for key in df_list.keys():
    df_list[key] = remove_missing_zero_data(df_list[key])
    print(f"{key} \t {df_list[key].shape}")

cnv 	 (24776, 579)
mrna 	 (12042, 593)
rnaseq 	 (20318, 308)
methyl 	 (21666, 616)
cnv_1 	 (24776, 555)
mrna_1 	 (12042, 555)
methyl_1 	 (21694, 555)
cnv_2 	 (24776, 292)
rnaseq_2 	 (20318, 292)
methyl_2 	 (22528, 292)


## Normalisation

In [10]:
def min_max_normalization(df):
    for column in df.columns:
        df[column] = (df[column] - df[column].min())/ (df[column].max() - df[column].min())
    return df

In [11]:
for key in df_list.keys():
    df_list[key] = min_max_normalization(df_list[key])

## Concatenation for multi-omics data

In [12]:
final_list = {}

final_list["mrna_methyl"] = pd.concat([df_list["mrna_1"], df_list["methyl_1"]], axis=0, join="inner")
final_list["methyl_cnv"] = pd.concat([df_list["cnv_1"], df_list["methyl_1"]], axis=0, join="inner")
final_list["cnv_mrna"] = pd.concat([df_list["cnv_1"], df_list["mrna_1"]], axis=0, join="inner")

final_list["methyl_rnaseq"] = pd.concat([df_list["methyl_2"], df_list["rnaseq_2"]], axis=0, join="inner")
final_list["cnv_rnaseq"] = pd.concat([df_list["cnv_2"], df_list["rnaseq_2"]], axis=0, join="inner")

final_list["cnv_methyl_mrna"] = pd.concat([final_list["methyl_cnv"], df_list["mrna_1"]], axis=0, join="inner")
final_list["cnv_methyl_rnaseq"] = pd.concat([final_list["methyl_rnaseq"], df_list["cnv_2"]], axis=0, join="inner")


for key in final_list.keys():
    print(f"{key} \t {final_list[key].shape}")

mrna_methyl 	 (33736, 555)
methyl_cnv 	 (46470, 555)
cnv_mrna 	 (36818, 555)
methyl_rnaseq 	 (42846, 292)
cnv_rnaseq 	 (45094, 292)
cnv_methyl_mrna 	 (58512, 555)
cnv_methyl_rnaseq 	 (67622, 292)


In [13]:
for key in final_list.keys():
    final_list[key].to_csv(f'./1_csv_data/{key}.csv')

In [14]:
for key in df_list.keys():
    df_list[key].to_csv(f'./1_csv_data/{key}.csv')